In [1]:
from __future__ import unicode_literals, print_function, division
import random
import time

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import rnn_utils

SOS_token, EOS_token = rnn_utils.SOS_token, rnn_utils.EOS_token
MAX_LENGTH = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
dataset = rnn_utils.prepareData('eng', 'cmn', MAX_LENGTH, device, True)
print(dataset.randomPair())

Reading lines...
Read 20133 sentence pairs
Trimmed to 10051 sentence pairs
Counting words...
Counted words:
cmn 2591
eng 4116
['你 喜 欢 这 本 书 吗 ？', 'Do you like this book ?']


In [3]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [4]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [5]:
class Trainer():
  
  def __init__(self, encoder, decoder, dataset, teacher_forcing_ratio=0.5, learning_rate=0.01, max_length=MAX_LENGTH):
    self.encoder = encoder
    self.decoder = decoder
    self.max_length = max_length
    self.teacher_forcing_ratio = teacher_forcing_ratio
    self.dataset = dataset

    self.encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    self.decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    self.criterion = nn.NLLLoss()
  
  def train(self, n_iters, print_every=1000):
    start = time.time()
    print_loss_total = 0  # Reset every print_every

    training_pairs = [dataset.random() for i in range(n_iters)]
    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor, target_tensor = training_pair[0], training_pair[1]

        loss = self.trainStep(input_tensor, target_tensor)
        print_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (rnn_utils.timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

  def trainStep(self, input_tensor, target_tensor):
    loss = 0

    self.encoder_optimizer.zero_grad()
    self.decoder_optimizer.zero_grad()

    encoder_hidden = self.encoder.initHidden()
    encoder_outputs = torch.zeros(self.max_length, self.encoder.hidden_size, device=device)

    input_length, target_length = input_tensor.size(0), target_tensor.size(0)
    for ei in range(input_length):
        encoder_output, encoder_hidden = self.encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    def get_decoder_input(di, decoder_output):
      if random.random() < self.teacher_forcing_ratio:
        # Teacher forcing: Feed the target as the next input
        return target_tensor[di]
      else:
        # Without teacher forcing: use its own predictions as the next input
        topv, topi = decoder_output.topk(1)
        return topi.squeeze().detach()

    for di in range(target_length):
        decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
        loss += self.criterion(decoder_output, target_tensor[di])
        decoder_input = get_decoder_input(di, decoder_output)
        if decoder_input.item() == EOS_token:
            break

    loss.backward()

    self.encoder_optimizer.step()
    self.decoder_optimizer.step()

    return loss.item() / target_length
    


In [6]:
class Evaluator():

  def __init__(self, encoder, decoder, dataset, max_length=MAX_LENGTH):
    self.decoder = decoder
    self.encoder = encoder
    self.max_length = max_length
    self.dataset = dataset

  def evaluateRandomly(self, n=10):
    for i in range(n):
        pair = dataset.randomPair()
        print('>', pair[0])
        print('=', pair[1])
        output_words = self.evaluate(pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')
    
  def evaluate(self, sentence):
    with torch.no_grad():
        input_tensor = dataset.sentenceToTensorFromInputLang(sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = self.encoder.initHidden()

        encoder_outputs = torch.zeros(self.max_length, self.encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = self.encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        decoder_hidden = encoder_hidden
        decoded_words = []

        for di in range(self.max_length):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)

            word = '<EOS>' if topi.item() == EOS_token else self.dataset.wordFromOutputLang(topi.item())
            decoded_words.append(word)

            if word == '<EOS>':
                break

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [7]:
hidden_size = 256
encoder = EncoderRNN(dataset.input_lang.n_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, dataset.output_lang.n_words).to(device)

trainer = Trainer(encoder, decoder, dataset)
trainer.train(1000, print_every=100)

0m 4s (- 0m 43s) (100 10%) 5.1823
0m 9s (- 0m 37s) (200 20%) 4.8750
0m 14s (- 0m 32s) (300 30%) 5.0956
0m 18s (- 0m 28s) (400 40%) 4.9604
0m 23s (- 0m 23s) (500 50%) 4.9862
0m 28s (- 0m 18s) (600 60%) 4.5788
0m 32s (- 0m 13s) (700 70%) 4.6331
0m 37s (- 0m 9s) (800 80%) 4.6542
0m 41s (- 0m 4s) (900 90%) 4.5740
0m 46s (- 0m 0s) (1000 100%) 4.5048


In [8]:
encoder.load_state_dict(torch.load('rnn-encoder-seq2seq.pkl'))
decoder.load_state_dict(torch.load('rnn-decoder-seq2seq.pkl'))

In [9]:
evaluator = Evaluator(encoder, decoder, dataset)
evaluator.evaluateRandomly()

> 她 反 應 很 快 。
= She has a very quick mind .
< She is very a a a . . <EOS>

> 我 父 親 愛 吃 比 薩 。
= My father loves pizza .
< My father loves pizza . . <EOS>

> 很 快 就 要 三 點 鐘 了 。
= It ll soon be three o clock .
< It ll soon be six o clock . <EOS>

> 你 写 过 博 客 吗 ？
= Have you ever written a blog ?
< Do you ever written a ? <EOS>

> 湯 姆 應 該 要 被 開 除 。
= Tom should be fired .
< Tom should be fired . <EOS>

> 他 们 用 肥 皂 洗 手 。
= They wash their hands with soap .
< They wash their hands with their hands . <EOS>

> 這 與 你 無 關 。
= This has nothing to do with you .
< It s nothing to do you . <EOS>

> 你 看 起 来 很 紧 张 。
= You look tense .
< You seem very nervous . <EOS>

> 他 把 我 們 弄 糊 塗 了 。
= He confused us .
< He made to us . <EOS>

> 我 要 幫 你 營 救 湯 姆 。
= I m going to help you rescue Tom .
< I m going to see you Tom . <EOS>



In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        # 略过部分代码

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)